In [1]:
from nltk.lm.preprocessing import padded_everygram_pipeline
import nltk
import codecs
import re
import random
from nltk.lm.preprocessing import pad_both_ends
from nltk import ngrams

In [2]:
text = codecs.open("/home/katty/anaconda3/_nlp/project_hp/corp/5.txt", "r", "utf-8").read()

In [3]:
sent =  [nltk.tokenize.word_tokenize(re.sub( '[^А-Яа-я]', " ", sent.lower())) for sent in nltk.tokenize.sent_tokenize(text)]
random.shuffle(sent)
train_sent = sent[:round(0.8*len(sent))]
validation_sent = sent[round(0.8*len(sent)):round(0.9*len(sent))]
test_sent = sent[round(0.9*len(sent))]

ngr = []
for elem in validation_sent[:5]:
    ngr += list(ngrams(pad_both_ends(elem, 2), 2))
print(ngr)

[('<s>', 'значит'), ('значит', 'вам'), ('вам', 'это'), ('это', 'неизвестно'), ('неизвестно', '</s>'), ('<s>', 'нам'), ('нам', 'бы'), ('бы', 'сюда'), ('сюда', 'боггарта'), ('боггарта', 'или'), ('или', 'кого'), ('кого', 'нибудь'), ('нибудь', 'вроде'), ('вроде', 'него'), ('него', 'я'), ('я', 'сам'), ('сам', 'впервые'), ('впервые', 'вызвал'), ('вызвал', 'патронуса'), ('патронуса', 'когда'), ('когда', 'боггарт'), ('боггарт', 'притворился'), ('притворился', 'дементором'), ('дементором', 'и'), ('и', 'благодаря'), ('благодаря', 'этому'), ('этому', 'научился'), ('научился', 'но'), ('но', 'это'), ('это', 'было'), ('было', 'бы'), ('бы', 'страшно'), ('страшно', '</s>'), ('<s>', 'ребята'), ('ребята', 'выросшие'), ('выросшие', 'в'), ('в', 'семьях'), ('семьях', 'волшебников'), ('волшебников', 'с'), ('с', 'детства'), ('детства', 'знали'), ('знали', 'имена'), ('имена', 'этих'), ('этих', 'пожирателей'), ('пожирателей', 'смерти'), ('смерти', 'их'), ('их', 'произносили'), ('произносили', 'почти'), ('почти

In [4]:
n = 2

In [5]:
from nltk.lm import Lidstone

def lindstone(lambda_):
    n_grams, words = padded_everygram_pipeline(n, train_sent)
    model = Lidstone(lambda_, n)
    model.fit(n_grams, words)
    return model

model_1 = lindstone(1)
model_0_1 = lindstone(0.1)
print(model_1.score('поттер', ['гарри']), model_0_1.score('поттер', ['гарри']))


0.0009530382201189655 0.004851938185271519


In [6]:
from nltk.lm import MLE


def mle():
    n_grams, words = padded_everygram_pipeline(n, train_sent)
    model = MLE(n)
    model.fit(n_grams, words)
    return model

model_mle = mle()
print(model_mle.score('поттер', ['гарри']))


0.0091683038637852


In [7]:
from nltk.lm import KneserNeyInterpolated

def kneser(lambda_):
    model = KneserNeyInterpolated(n, lambda_)
    n_grams, words = padded_everygram_pipeline(n, train_sent)
    model.fit(n_grams, words)
    return model

model_kneser = kneser(0.1)
print(model_kneser.score('поттер', ['гарри']))

0.009136678298989573


In [14]:
max_perplexity = float('-inf')
best_model = ""
best_lambda = 0

all_sentence_n_grams = []
for elem in validation_sent:
    all_sentence_n_grams += list(ngrams(pad_both_ends(elem, 2), 2))

In [9]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ngrams


for lambda_ in [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5]:

        model = lindstone(lambda_)
        perplexity = model.perplexity(all_sentence_n_grams)
        print("Lindstone: lambda =", lambda_, "perplexity=", perplexity)
        if perplexity > max_perplexity:
            max_perplexity = perplexity
            best_lambda = lambda_

            best_model = model

Lindstone: lambda = 0.01 perplexity= 2685.5809310619297
Lindstone: lambda = 0.05 perplexity= 3493.1306357638664
Lindstone: lambda = 0.1 perplexity= 4182.676035354206
Lindstone: lambda = 0.2 perplexity= 5170.664898946924
Lindstone: lambda = 0.5 perplexity= 7058.883682229276
Lindstone: lambda = 1 perplexity= 8983.982291309027
Lindstone: lambda = 2 perplexity= 11306.398123224297
Lindstone: lambda = 5 perplexity= 14770.762721619145


In [18]:
for lambda_ in [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 0.95, 0.99]:
    all_sentence_n_grams = []
    for elem in validation_sent:
        all_sentence_n_grams += list(ngrams(pad_both_ends(elem, 2), 2))
    print(all_sentence_n_grams)
    model = kneser(lambda_)
    perplexity = model.perplexity(all_sentence_n_grams)
    print("KneyserNey: lambda =", lambda_, "perplexity=", perplexity)
    if perplexity > max_perplexity:
        max_perplexity = perplexity
        best_lambda = lambda_
        best_model = model

[('<s>', 'значит'), ('значит', 'вам'), ('вам', 'это'), ('это', 'неизвестно'), ('неизвестно', '</s>'), ('<s>', 'нам'), ('нам', 'бы'), ('бы', 'сюда'), ('сюда', 'боггарта'), ('боггарта', 'или'), ('или', 'кого'), ('кого', 'нибудь'), ('нибудь', 'вроде'), ('вроде', 'него'), ('него', 'я'), ('я', 'сам'), ('сам', 'впервые'), ('впервые', 'вызвал'), ('вызвал', 'патронуса'), ('патронуса', 'когда'), ('когда', 'боггарт'), ('боггарт', 'притворился'), ('притворился', 'дементором'), ('дементором', 'и'), ('и', 'благодаря'), ('благодаря', 'этому'), ('этому', 'научился'), ('научился', 'но'), ('но', 'это'), ('это', 'было'), ('было', 'бы'), ('бы', 'страшно'), ('страшно', '</s>'), ('<s>', 'ребята'), ('ребята', 'выросшие'), ('выросшие', 'в'), ('в', 'семьях'), ('семьях', 'волшебников'), ('волшебников', 'с'), ('с', 'детства'), ('детства', 'знали'), ('знали', 'имена'), ('имена', 'этих'), ('этих', 'пожирателей'), ('пожирателей', 'смерти'), ('смерти', 'их'), ('их', 'произносили'), ('произносили', 'почти'), ('почти

ZeroDivisionError: float division by zero

In [ ]:
all_sentence_n_grams = []
for elem in test_sent:
    all_sentence_n_grams += list(ngrams(pad_both_ends(elem, 2), 2))

if best_model == "lind":
    print("Кажется Lindstone выиграл, perplexity на validation set", max_perplexity)


    test_perplexity = best_model.perplexity(all_sentence_n_grams)
    print("Perplexity на test set:", test_perplexity)
if best_model == "kn":
    print("Кажется Lindstone выиграл, perplexity на validation set", max_perplexity)

    test_perplexity = best_model.perplexity(all_sentence_n_grams)
    print("Perplexity на test set:", test_perplexity)